In [1]:
import requests
import utils

year = 2021  # 2020 and 2021 available

user_id = {2021: 1075819, 2020: 1066392}
url = f'https://adventofcode.com/{year}/leaderboard/private/view/{user_id[year]}.json'

request = requests.get(url, cookies=utils.get_cookies())
data = request.json()

In [2]:
import pandas as pd

df = pd.json_normalize(data['members'].values())
df = df[['name', 'local_score', 'stars'] + list(sorted(df.columns[6:], key=lambda x: float(x[21:-12])))]
df.columns = ['name', 'score' ,'stars'] + [col[21:-12] for col in df.columns[3:]]

local_time = + 1 # CEST

acc_times = pd.DataFrame(data=df['name'])
df['accumulated_time'] = pd.Timedelta(0)
for i in range(3, df.shape[1]-1):
    df[df.columns[i]] = pd.to_datetime(df[df.columns[i]], unit='s') + pd.Timedelta(local_time, unit='H')
    if i%2 == 0:
        df[df.columns[i]] -= df[df.columns[i-1]]
        df['accumulated_time'] += df[df.columns[i]]
        
        day = df.columns[i].split('.')[0]
        prev_day = str(int(day)-1)
        acc_times[day] = df[df.columns[i]] if prev_day not in acc_times else acc_times[prev_day] + df[df.columns[i]]

df = df.sort_values('score', ascending=False)

def style_data(x):
    if isinstance(x, pd.Timestamp):
        return x.strftime('%H:%M:%S')
    if isinstance(x, pd.Timedelta):
        if x > pd.Timedelta(1, 'H'):
            return str(x)[-8:]
        else:
            return str(x)[-5:]
    return x

df[df.stars>0].style.format(style_data).hide_index()

name,score,stars,1.1,1.2,2.1,2.2,3.1,3.2,4.1,4.2,5.1,5.2,accumulated_time
Arnau Q,134,10,08:56:48,02:38,08:17:02,03:07,09:44:49,30:14,10:07:12,11:57,09:28:56,20:05,01:08:01
Andreu Vall Hernàndez,124,10,08:24:30,07:44,09:11:17,02:02,12:33:15,27:39,10:01:43,09:16,11:17:50,08:50,55:31
albertsalles4,120,10,10:22:07,08:49,09:21:50,02:31,10:10:11,15:24,11:54:16,12:10,10:26:11,09:42,48:36
David Rial Figols,110,10,10:36:31,08:55,09:31:31,02:43,07:09:12,36:45,16:58:57,06:52,11:25:20,23:36,01:18:51
Arnau Martínez Tomàs,94,10,23:53:14,45:12,10:23:49,25:47,12:59:36,30:52,11:25:20,12:41,12:45:06,23:25,02:17:57
Antonio Tigri,93,10,15:22:36,12:46,10:56:21,03:52,11:37:32,34:25,12:27:54,17:41,13:36:21,23:21,01:32:05
EstherRH00,84,10,13:11:40,42:39,21:05:03,03:25,13:39:55,23:11,13:20:28,16:00,13:01:04,20:08,01:45:23
sarabase,67,8,18:32:58,18:29,12:06:40,03:32,11:23:58,55:36,17:37:14,06:45,NaT,NaT,NaT


In [5]:
from datetime import datetime
import plotly.offline as po
import plotly.express as px

n_days = int(df.columns[-2].split('.')[0])
too_far = datetime.strptime(f'2/12/{year} 06', '%d/%m/%Y %H')

times_data = []
for _, row in df.iterrows():
    for day in range(1, n_days+1):
        date = row[f'{day}.1']
        if not pd.isnull(date):
            time = date-pd.Timedelta(day-1, unit='d')
            if time < too_far:
                times_data.append({'name': row['name'], 'day': day, 'time': time})

times = pd.DataFrame(times_data)

fig = px.line(times, x='day', y='time', line_group='name', color='name', markers=True, title='Hour when Part One Solved')
fig.update_yaxes(tickformat='%H:%M')
fig.update_xaxes(dtick=1)

po.iplot(fig)

In [6]:
base = datetime.strptime('1/12/21', '%d/%m/%y')

times_data = []
for _, row in df.iterrows():
    for day in range(1, n_days+1):
        time = row[f'{day}.2']
        if not pd.isnull(time):
            times_data.append({'name': row['name'], 'day': day, 'time': time+base})

times = pd.DataFrame(times_data)

fig = px.line(times, x='day', y='time', line_group='name', color='name', markers=True, title='Second Part Spent Time')

fig.update_yaxes(tickformat='%H:%M:%S')
fig.update_xaxes(dtick=1)

po.iplot(fig)

In [7]:
times_data = []
for _, row in acc_times.iterrows():
    for day in range(1, n_days+1):
        time = row[str(day)]
        if not pd.isnull(time):
            times_data.append({'name': row['name'], 'day': day, 'time': time+base})

times = pd.DataFrame(times_data)

fig = px.line(times, x='day', y='time', line_group='name', color='name', markers=True, title='Accumulated Second Part Spent Time')

fig.update_yaxes(tickformat='%H:%M')
fig.update_xaxes(dtick=1)

po.iplot(fig)